In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
from datetime import datetime
from sqlalchemy import create_engine
from sqlalchemy import Table, Column, Integer, String, Float, MetaData
import json
from config import password

In [2]:
# pull in percent of safe water csv
safe_water_file = "data/percent_safe_water.csv"
safe_water_df = pd.read_csv(safe_water_file)
safe_water_df


,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,Unnamed: 65
0,Aruba,ABW,People using safely managed drinking water ser...,SH.H2O.SMDW.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,AFG,People using safely managed drinking water ser...,SH.H2O.SMDW.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Angola,AGO,People using safely managed drinking water ser...,SH.H2O.SMDW.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Albania,ALB,People using safely managed drinking water ser...,SH.H2O.SMDW.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,65.901394,67.893339,69.903702,69.944500,69.984329,70.023150,NaN,NaN,NaN,NaN
4,Andorra,AND,People using safely managed drinking water ser...,SH.H2O.SMDW.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,90.640001,90.640000,90.639998,90.640002,90.640000,90.639997,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259,Kosovo,XKX,People using safely managed drinking water ser...,SH.H2O.SMDW.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
260,"Yemen, Rep.",YEM,People using safely managed drinking water ser...,SH.H2O.SMDW.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
261,South Africa,ZAF,People using safely managed drinking water ser...,SH.H2O.SMDW.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
262,Zambia,ZMB,People using safely managed drinking water ser...,SH.H2O.SMDW.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
#select columns for analysis
safe_water_df = safe_water_df[['Country Name', 'Country Code','2017']]
safe_water_df

,Country Name,Country Code,2017
0,Aruba,ABW,NaN
1,Afghanistan,AFG,NaN
2,Angola,AGO,NaN
3,Albania,ALB,70.023150
4,Andorra,AND,90.639997
...,...,...,...
259,Kosovo,XKX,NaN
260,"Yemen, Rep.",YEM,NaN
261,South Africa,ZAF,NaN
262,Zambia,ZMB,NaN


In [6]:
#convert strings to lower case for PG Admin connection
safe_water_df.columns = safe_water_df.columns.str.lower()
safe_water_df.columns = safe_water_df.columns.str.replace(' ','_')
safe_water_df

,country_name,code,safe_water_2017
0,Aruba,ABW,NaN
1,Afghanistan,AFG,NaN
2,Angola,AGO,NaN
3,Albania,ALB,70.023150
4,Andorra,AND,90.639997
...,...,...,...
259,Kosovo,XKX,NaN
260,"Yemen, Rep.",YEM,NaN
261,South Africa,ZAF,NaN
262,Zambia,ZMB,NaN


In [7]:
#rename columns 
safe_water_df = safe_water_df.rename(columns={'country_code': 'code', '2017':'safe_water_2017'})
safe_water_df

,country_name,code,safe_water_2017
0,Aruba,ABW,NaN
1,Afghanistan,AFG,NaN
2,Angola,AGO,NaN
3,Albania,ALB,70.023150
4,Andorra,AND,90.639997
...,...,...,...
259,Kosovo,XKX,NaN
260,"Yemen, Rep.",YEM,NaN
261,South Africa,ZAF,NaN
262,Zambia,ZMB,NaN


In [8]:
#drop NaN columns
safe_water_df = safe_water_df.dropna()
safe_water_df

,country_name,code,safe_water_2017
3,Albania,ALB,70.023150
4,Andorra,AND,90.639997
8,Armenia,ARM,86.476355
9,American Samoa,ASM,12.575083
12,Austria,AUT,98.906201
...,...,...,...
249,United States,USA,99.001778
250,Uzbekistan,UZB,58.903805
256,Vanuatu,VUT,44.130326
257,World,WLD,70.644880


In [9]:
#check data types (need to convert object to float)
safe_water_df.dtypes

country_name        object
code                object
safe_water_2017    float64
dtype: object

In [10]:
# pull in death by risk factor
cause_of_death_file = "data/number_of_deaths_by_risk_factor.csv"
cod_df = pd.read_csv(cause_of_death_file)
cod_df

,Entity,Code,Year,Deaths - Unsafe water source - Sex: Both - Age: All Ages (Number),Deaths - Unsafe sanitation - Sex: Both - Age: All Ages (Number),Deaths - No access to handwashing facility - Sex: Both - Age: All Ages (Number),Deaths - Household air pollution from solid fuels - Sex: Both - Age: All Ages (Number),Deaths - Non-exclusive breastfeeding - Sex: Both - Age: All Ages (Number),Deaths - Discontinued breastfeeding - Sex: Both - Age: All Ages (Number),Deaths - Child wasting - Sex: Both - Age: All Ages (Number),...,Deaths - Air pollution - Sex: Both - Age: All Ages (Number),Deaths – Outdoor air pollution (all ages) (IHME),Deaths - Diet low in fiber - Sex: Both - Age: All Ages (Number),Deaths - Diet high in sodium - Sex: Both - Age: All Ages (Number),Deaths - Diet low in legumes - Sex: Both - Age: All Ages (Number),Deaths - Diet low in calcium - Sex: Both - Age: All Ages (Number),Deaths - Diet high in red meat - Sex: Both - Age: All Ages (Number),Deaths - Diet low in whole grains - Sex: Both - Age: All Ages (Number),Deaths - Diet low in nuts and seeds - Sex: Both - Age: All Ages (Number),Deaths - Diet low in seafood omega-3 fatty acids - Sex: Both - Age: All Ages (Number)
0,Afghanistan,AFG,1990,7554.049543,5887.747628,5412.314513,22388.497233,3221.138842,156.097553,22778.849249,...,26598.006727,4383.83,2864.234512,2737.197934,3418.965194,181.287701,0.220484,11381.377345,7299.867330,6456.565238
1,Afghanistan,AFG,1991,7359.676749,5732.770160,5287.891103,22128.758206,3150.559597,151.539851,22292.691113,...,26379.532218,4426.36,2974.479775,2741.184956,3457.398250,184.251584,0.184714,11487.832390,7386.764303,6511.344276
2,Afghanistan,AFG,1992,7650.437822,5954.804987,5506.657363,22873.768789,3331.349048,156.609194,23102.197940,...,27263.127914,4568.91,3168.591633,2798.560245,3586.490291,191.318450,0.151354,11866.235572,7640.628526,6703.095594
3,Afghanistan,AFG,1993,10270.731383,7986.736613,7104.620351,25599.756284,4477.006100,206.834451,27902.669960,...,30495.561499,5080.29,3401.045056,2853.301679,3746.266658,199.767293,0.109392,12335.961682,7968.311853,6946.196824
4,Afghanistan,AFG,1994,11409.177112,8863.010065,8051.515953,28013.167200,5102.622054,233.930571,32929.005932,...,33323.161401,5499.23,3599.709735,2880.025765,3869.288402,206.619714,0.059103,12672.950191,8244.368430,7138.030745
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6463,Zimbabwe,ZWE,2013,4254.282075,2977.649750,3913.210510,7613.561005,1037.968042,59.150493,7703.062474,...,9593.033931,2053.58,619.391594,1018.389001,469.787520,182.630009,0.000311,2687.636261,2409.930182,1757.282549
6464,Zimbabwe,ZWE,2014,4098.769691,2856.426187,3809.245683,7429.446352,972.886327,54.334796,7401.059382,...,9387.193480,2030.92,604.168238,1016.407438,482.800746,184.379579,0.000756,2654.381923,2399.261581,1740.938755
6465,Zimbabwe,ZWE,2015,3921.291358,2717.735794,3688.442102,7267.029297,912.248164,50.255551,7100.476546,...,9189.336702,1994.91,594.347147,1019.971539,494.337563,188.020716,0.001512,2635.950107,2398.525219,1732.926445
6466,Zimbabwe,ZWE,2016,3802.257512,2624.315858,3603.179799,7134.595677,875.706009,47.719473,6823.766727,...,9092.577378,2030.88,592.092457,1032.181216,507.088683,192.913319,0.002787,2641.376815,2417.422521,1740.653975


In [11]:
#select only year 2017 (most recent data)
cod_df = cod_df[cod_df.Year == 2017]
cod_df

,Entity,Code,Year,Deaths - Unsafe water source - Sex: Both - Age: All Ages (Number),Deaths - Unsafe sanitation - Sex: Both - Age: All Ages (Number),Deaths - No access to handwashing facility - Sex: Both - Age: All Ages (Number),Deaths - Household air pollution from solid fuels - Sex: Both - Age: All Ages (Number),Deaths - Non-exclusive breastfeeding - Sex: Both - Age: All Ages (Number),Deaths - Discontinued breastfeeding - Sex: Both - Age: All Ages (Number),Deaths - Child wasting - Sex: Both - Age: All Ages (Number),...,Deaths - Air pollution - Sex: Both - Age: All Ages (Number),Deaths – Outdoor air pollution (all ages) (IHME),Deaths - Diet low in fiber - Sex: Both - Age: All Ages (Number),Deaths - Diet high in sodium - Sex: Both - Age: All Ages (Number),Deaths - Diet low in legumes - Sex: Both - Age: All Ages (Number),Deaths - Diet low in calcium - Sex: Both - Age: All Ages (Number),Deaths - Diet high in red meat - Sex: Both - Age: All Ages (Number),Deaths - Diet low in whole grains - Sex: Both - Age: All Ages (Number),Deaths - Diet low in nuts and seeds - Sex: Both - Age: All Ages (Number),Deaths - Diet low in seafood omega-3 fatty acids - Sex: Both - Age: All Ages (Number)
27,Afghanistan,AFG,2017,5.256649e+03,3783.111117,4156.209013,1.939738e+04,2422.369175,94.118135,1.457527e+04,...,2.605420e+04,6872.06,4889.502247,3.457084e+03,4384.887948,325.722066,9.994555e-04,1.488496e+04,9.074388e+03,7.775130e+03
55,Albania,ALB,2017,4.091664e+00,1.597907,4.501301,7.368815e+02,8.173119,0.071829,4.061321e+01,...,1.638571e+03,919.59,428.785803,1.668336e+03,429.775831,23.218640,2.710364e+00,2.206717e+03,1.130544e+03,8.457940e+02
83,Algeria,DZA,2017,1.899361e+02,58.910622,191.730462,5.811236e+01,126.552097,7.723609,6.354547e+02,...,1.322228e+04,13315.30,3004.104009,5.828760e+03,2761.644707,370.146859,8.200700e-03,1.610308e+04,1.148331e+04,8.810311e+03
111,American Samoa,ASM,2017,7.200565e-01,0.179205,0.293769,7.550280e+00,0.118482,0.003936,2.285860e+00,...,2.217993e+01,14.63,2.679124,8.519993e+00,4.836963,1.089152,3.212988e-01,2.563875e+01,1.752416e+01,8.137023e+00
139,Andean Latin America,NaN,2017,1.257987e+03,733.649228,1578.129626,5.773893e+03,344.102079,20.477721,3.615349e+03,...,1.768615e+04,12011.09,3337.535571,9.758082e+03,2169.986257,1033.264841,5.867989e+01,1.096710e+04,8.049420e+03,4.788967e+03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6355,Western Sub-Saharan Africa,NaN,2017,2.324944e+05,160952.618364,169944.465538,1.721336e+05,53606.767487,3448.963539,3.461004e+05,...,2.536535e+05,82793.37,6408.003926,2.474591e+04,3056.112541,3461.469135,1.779343e+00,6.201625e+04,3.404690e+04,2.969483e+04
6383,World,OWID_WRL,2017,1.232368e+06,774240.986150,707247.712854,1.640600e+06,160983.374426,10011.809235,1.077102e+06,...,4.895476e+06,3408877.62,873407.776902,3.196514e+06,534767.131156,184760.116730,2.483329e+04,3.065589e+06,2.062522e+06,1.444907e+06
6411,Yemen,YEM,2017,6.435590e+03,4011.286680,3029.543803,4.700403e+03,2440.200553,237.628511,9.475333e+03,...,1.237858e+04,7777.21,3730.797892,3.271575e+03,2254.400766,274.632071,1.044604e-07,1.244528e+04,8.677625e+03,6.783494e+03
6439,Zambia,ZMB,2017,6.288668e+03,4718.900370,4553.893978,5.452029e+03,752.156871,63.183108,7.289651e+03,...,7.105757e+03,1726.75,397.244094,1.183670e+03,54.161681,189.611543,1.473957e-07,1.792215e+03,1.580105e+03,1.163538e+03


In [12]:
#sum all deaths
cod_sum = cod_df.sum(axis=1)
cod_sum

27      2.815395e+05
55      3.314386e+04
83      2.377406e+05
111     2.456121e+03
139     2.576397e+05
            ...     
6355    3.050567e+06
6383    6.687848e+07
6411    2.107088e+05
6439    1.055452e+05
6467    1.229190e+05
Length: 231, dtype: float64

In [13]:
#add in sum column (https://www.geeksforgeeks.org/adding-new-column-to-existing-dataframe-in-pandas/)
cod_df['sum'] = cod_sum
cod_df.head()

<ipython-input-13-e82c2f813afb>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cod_df['sum'] = cod_sum


,Entity,Code,Year,Deaths - Unsafe water source - Sex: Both - Age: All Ages (Number),Deaths - Unsafe sanitation - Sex: Both - Age: All Ages (Number),Deaths - No access to handwashing facility - Sex: Both - Age: All Ages (Number),Deaths - Household air pollution from solid fuels - Sex: Both - Age: All Ages (Number),Deaths - Non-exclusive breastfeeding - Sex: Both - Age: All Ages (Number),Deaths - Discontinued breastfeeding - Sex: Both - Age: All Ages (Number),Deaths - Child wasting - Sex: Both - Age: All Ages (Number),...,Deaths – Outdoor air pollution (all ages) (IHME),Deaths - Diet low in fiber - Sex: Both - Age: All Ages (Number),Deaths - Diet high in sodium - Sex: Both - Age: All Ages (Number),Deaths - Diet low in legumes - Sex: Both - Age: All Ages (Number),Deaths - Diet low in calcium - Sex: Both - Age: All Ages (Number),Deaths - Diet high in red meat - Sex: Both - Age: All Ages (Number),Deaths - Diet low in whole grains - Sex: Both - Age: All Ages (Number),Deaths - Diet low in nuts and seeds - Sex: Both - Age: All Ages (Number),Deaths - Diet low in seafood omega-3 fatty acids - Sex: Both - Age: All Ages (Number),sum
27,Afghanistan,AFG,2017,5256.649276,3783.111117,4156.209013,19397.378213,2422.369175,94.118135,14575.269211,...,6872.06,4889.502247,3457.083571,4384.887948,325.722066,0.000999,14884.958403,9074.388004,7775.129667,281539.488500
55,Albania,ALB,2017,4.091664,1.597907,4.501301,736.881455,8.173119,0.071829,40.613209,...,919.59,428.785803,1668.336331,429.775831,23.218640,2.710364,2206.716542,1130.544001,845.793999,33143.859240
83,Algeria,DZA,2017,189.936097,58.910622,191.730462,58.112360,126.552097,7.723609,635.454715,...,13315.30,3004.104009,5828.759675,2761.644707,370.146859,0.008201,16103.078982,11483.306530,8810.310800,237740.565032
111,American Samoa,ASM,2017,0.720056,0.179205,0.293769,7.550280,0.118482,0.003936,2.285860,...,14.63,2.679124,8.519993,4.836963,1.089152,0.321299,25.638747,17.524164,8.137023,2456.120959
139,Andean Latin America,NaN,2017,1257.987062,733.649228,1578.129626,5773.892670,344.102079,20.477721,3615.348894,...,12011.09,3337.535571,9758.081534,2169.986257,1033.264841,58.679893,10967.097020,8049.419977,4788.966632,257639.739148


In [14]:
#select columns for analysis
cod_df = cod_df[['Entity', 'Code','Year', 'Deaths - Unsafe water source - Sex: Both - Age: All Ages (Number)', 'Deaths - Unsafe sanitation - Sex: Both - Age: All Ages (Number)', 'Deaths - No access to handwashing facility - Sex: Both - Age: All Ages (Number)', 'sum']]
cod_df.head()

,Entity,Code,Year,Deaths - Unsafe water source - Sex: Both - Age: All Ages (Number),Deaths - Unsafe sanitation - Sex: Both - Age: All Ages (Number),Deaths - No access to handwashing facility - Sex: Both - Age: All Ages (Number),sum
27,Afghanistan,AFG,2017,5256.649276,3783.111117,4156.209013,281539.488500
55,Albania,ALB,2017,4.091664,1.597907,4.501301,33143.859240
83,Algeria,DZA,2017,189.936097,58.910622,191.730462,237740.565032
111,American Samoa,ASM,2017,0.720056,0.179205,0.293769,2456.120959
139,Andean Latin America,NaN,2017,1257.987062,733.649228,1578.129626,257639.739148


In [15]:
#rename columns 
cod_df = cod_df.rename(columns={'Deaths - Unsafe water source - Sex: Both - Age: All Ages (Number)': 'unsafe_water_source', 'Deaths - Unsafe sanitation - Sex: Both - Age: All Ages (Number)':'unsafe_sanitation', 'Deaths - No access to handwashing facility - Sex: Both - Age: All Ages (Number)':'no_access_handwashing'})
cod_df.head()

,Entity,Code,Year,unsafe_water_source,unsafe_sanitation,no_access_handwashing,sum
27,Afghanistan,AFG,2017,5256.649276,3783.111117,4156.209013,281539.488500
55,Albania,ALB,2017,4.091664,1.597907,4.501301,33143.859240
83,Algeria,DZA,2017,189.936097,58.910622,191.730462,237740.565032
111,American Samoa,ASM,2017,0.720056,0.179205,0.293769,2456.120959
139,Andean Latin America,NaN,2017,1257.987062,733.649228,1578.129626,257639.739148


In [16]:
cod_df = cod_df.dropna()
cod_df

,Entity,Code,Year,unsafe_water_source,unsafe_sanitation,no_access_handwashing,sum
27,Afghanistan,AFG,2017,5.256649e+03,3783.111117,4156.209013,2.815395e+05
55,Albania,ALB,2017,4.091664e+00,1.597907,4.501301,3.314386e+04
83,Algeria,DZA,2017,1.899361e+02,58.910622,191.730462,2.377406e+05
111,American Samoa,ASM,2017,7.200565e-01,0.179205,0.293769,2.456121e+03
167,Andorra,AND,2017,2.702152e-02,0.018488,0.206306,2.510520e+03
...,...,...,...,...,...,...,...
6271,Vietnam,VNM,2017,1.576620e+03,813.566332,1730.772182,7.354276e+05
6383,World,OWID_WRL,2017,1.232368e+06,774240.986150,707247.712854,6.687848e+07
6411,Yemen,YEM,2017,6.435590e+03,4011.286680,3029.543803,2.107088e+05
6439,Zambia,ZMB,2017,6.288668e+03,4718.900370,4553.893978,1.055452e+05


In [17]:
#convert strings to lower case for PG Admin connection
cod_df.columns = cod_df.columns.str.lower()
cod_df.head()

,entity,code,year,unsafe_water_source,unsafe_sanitation,no_access_handwashing,sum
27,Afghanistan,AFG,2017,5256.649276,3783.111117,4156.209013,281539.488500
55,Albania,ALB,2017,4.091664,1.597907,4.501301,33143.859240
83,Algeria,DZA,2017,189.936097,58.910622,191.730462,237740.565032
111,American Samoa,ASM,2017,0.720056,0.179205,0.293769,2456.120959
167,Andorra,AND,2017,0.027022,0.018488,0.206306,2510.519621


In [18]:
#find percentage of deaths (https://stackoverflow.com/questions/36619631/how-to-divide-two-columns-element-wise-in-a-pandas-dataframe)
cod_df["unsafe_water_perct"] = (cod_df['unsafe_water_source']/cod_df['sum'])*100
cod_df["unsafe_sanitation_perct"] = (cod_df['unsafe_sanitation']/cod_df['sum'])*100
cod_df["no_handwashing_perct"] = (cod_df['no_access_handwashing']/cod_df['sum'])*100
cod_df.head()


,entity,code,year,unsafe_water_source,unsafe_sanitation,no_access_handwashing,sum,unsafe_water_perct,unsafe_sanitation_perct,no_handwashing_perct
27,Afghanistan,AFG,2017,5256.649276,3783.111117,4156.209013,281539.488500,1.867109,1.343723,1.476244
55,Albania,ALB,2017,4.091664,1.597907,4.501301,33143.859240,0.012345,0.004821,0.013581
83,Algeria,DZA,2017,189.936097,58.910622,191.730462,237740.565032,0.079892,0.024779,0.080647
111,American Samoa,ASM,2017,0.720056,0.179205,0.293769,2456.120959,0.029317,0.007296,0.011961
167,Andorra,AND,2017,0.027022,0.018488,0.206306,2510.519621,0.001076,0.000736,0.008218


In [19]:
#check data types (need to convert object to float)
cod_df.dtypes

entity                      object
code                        object
year                         int64
unsafe_water_source        float64
unsafe_sanitation          float64
no_access_handwashing      float64
sum                        float64
unsafe_water_perct         float64
unsafe_sanitation_perct    float64
no_handwashing_perct       float64
dtype: object

In [22]:
merge_df = pd.merge(cod_df, safe_water_df, on="code")
merge_df

,entity,code,year,unsafe_water_source,unsafe_sanitation,no_access_handwashing,sum,unsafe_water_perct,unsafe_sanitation_perct,no_handwashing_perct,country_name,safe_water_2017
0,Albania,ALB,2017,4.091664,1.597907,4.501301,3.314386e+04,0.012345,0.004821,0.013581,Albania,70.023150
1,American Samoa,ASM,2017,0.720056,0.179205,0.293769,2.456121e+03,0.029317,0.007296,0.011961,American Samoa,12.575083
2,Andorra,AND,2017,0.027022,0.018488,0.206306,2.510520e+03,0.001076,0.000736,0.008218,Andorra,90.639997
3,Armenia,ARM,2017,3.045523,4.379285,12.664885,4.578739e+04,0.006651,0.009564,0.027660,Armenia,86.476355
4,Austria,AUT,2017,2.407016,6.282778,8.644877,9.225982e+04,0.002609,0.006810,0.009370,Austria,98.906201
...,...,...,...,...,...,...,...,...,...,...,...,...
95,Ukraine,UKR,2017,54.127601,41.273840,181.319152,1.234861e+06,0.004383,0.003342,0.014683,Ukraine,92.006857
96,United Kingdom,GBR,2017,39.975723,6.198679,252.575925,5.263808e+05,0.007594,0.001178,0.047983,United Kingdom,99.988779
97,United States,USA,2017,296.744882,576.851003,795.287168,2.902411e+06,0.010224,0.019875,0.027401,United States,99.001778
98,Uzbekistan,UZB,2017,105.053926,120.096099,296.505464,3.607610e+05,0.029120,0.033290,0.082189,Uzbekistan,58.903805


In [ ]:
# merge_df.to_csv(r'merge_df.csv', index = False)

In [23]:
merge_df.dtypes

entity                      object
code                        object
year                         int64
unsafe_water_source        float64
unsafe_sanitation          float64
no_access_handwashing      float64
sum                        float64
unsafe_water_perct         float64
unsafe_sanitation_perct    float64
no_handwashing_perct       float64
country_name                object
safe_water_2017            float64
dtype: object

In [24]:
# Creating our engine for PostGRES database
rds_connection_string = (f'postgres:{password}@localhost:5432/water_data_db')
engine = create_engine(f'postgresql://{rds_connection_string}')

In [25]:
meta = MetaData()
sanitation = Table(
   'sanitation', meta, 
   Column('entity', String), 
   Column('code', String, primary_key = True), 
   Column('year', Integer),
   Column('unsafe_water_source', Float),
   Column('unsafe_sanitation', Float),
   Column('no_access_handwashing', Float),
   Column('sum', Float),
   Column('unsafe_water_perct', Float),
   Column('unsafe_sanitation_perct', Float),
   Column('no_handwashing_perct', Float),
   Column('country_name', String),
   Column('safe_water_2017', Float) 
)
meta.create_all(engine)

In [29]:
#uploading the data to the SQL database sanitation
merge_df.to_sql(name='sanitation', con=engine, if_exists='append', index=False)
conn = engine.connect()

In [30]:
test= pd.read_sql("""select * from sanitation""", conn)

In [31]:
test.tail()

,entity,code,year,unsafe_water_source,unsafe_sanitation,no_access_handwashing,sum,unsafe_water_perct,unsafe_sanitation_perct,no_handwashing_perct,country_name,safe_water_2017
95,Ukraine,UKR,2017,54.127601,41.273840,181.319152,1.234861e+06,0.004383,0.003342,0.014683,Ukraine,92.006857
96,United Kingdom,GBR,2017,39.975723,6.198679,252.575925,5.263808e+05,0.007594,0.001178,0.047983,United Kingdom,99.988779
97,United States,USA,2017,296.744882,576.851003,795.287168,2.902411e+06,0.010224,0.019875,0.027401,United States,99.001778
98,Uzbekistan,UZB,2017,105.053926,120.096099,296.505464,3.607610e+05,0.029120,0.033290,0.082189,Uzbekistan,58.903805
99,Vanuatu,VUT,2017,28.056882,20.948589,16.391068,5.146171e+03,0.545199,0.407071,0.318510,Vanuatu,44.130326
